In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from source import Vermont_Farm
from statsmodels.compat import lzip
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
from stargazer.stargazer import Stargazer

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
VF = Vermont_Farm()
# df = pd.read_csv('./1850final_w_TotalOutput.csv')

# Revenue per Land as dependent variable

In [4]:
df_diversity = pd.read_csv('1860Midd_final.csv')
df_diversity = df_diversity.drop(['Name', 'town', 'unimproved', 'cash_val', 'produce_val', 'pg_num', 'year', 'rown'], axis=1)
df_diversity.head(1)

,improved,machine_val,horse,mules,milkcow,oxen,cattle,sheep,swine,livestock_val,...,flax,flaxseed,silk,maplesugar,canesugar,molasses,beewax,honey,manufactures_val,animals_val
0,38,50,2,0,1,0,4,0,0,210,...,0,0,0,0,0,0,0,0,15,28


In [5]:
df_diversity = df_diversity.to_numpy()
print(df_diversity.shape)
df_diversity = 1 * (df_diversity > 0)
print(df_diversity.shape)

(159, 44)
(159, 44)


In [6]:
df3 = pd.read_csv('./1860final_w_TotalOutput.csv')
div_score = np.sum(df_diversity, axis=1)
print(div_score.shape)
df3['diversity_score'] = div_score

(159,)


In [7]:
entropy_score = df3["entropy_score"].to_numpy()
df3['diversity_squared'] = np.square(div_score)
df3['e_to_diversity'] = np.exp(div_score)
df3['entropy_squared'] = np.square(entropy_score)

# generated diversity score

In [8]:
cols2 = ['improved', 'unimproved', 
       'machine_val', 'horse', 'mules']
# Fit regression model (using the natural log of one of the regressors)
# results = smf.ols("Lottery ~ Literacy + np.log(Pop1831)", data=dat).fit()
reg1 = smf.ols("revenue_per_land ~ diversity_score", data=df3).fit()
# print(reg1.summary())
reg2 = smf.ols("revenue_per_land ~ diversity_score + improved + unimproved", data=df3).fit()

In [9]:
reg3 = smf.ols("revenue_per_land ~ diversity_score + diversity_squared + improved + unimproved", data=df3).fit()
reg4 = smf.ols("revenue_per_land ~ entropy_score + improved + unimproved", data=df3).fit()
reg5 = smf.ols("revenue_per_land ~ entropy_score + entropy_squared + improved + unimproved", data=df3).fit()
reg6 = smf.ols("revenue_per_land ~ entropy_score + machine_val + cash_val", data=df3).fit()
reg7 = smf.ols("revenue_per_land ~ entropy_score + machine_val + cash_val + livestock_val + animals_val", data=df3).fit()

In [10]:
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:       revenue_per_land   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.03932
Date:                Wed, 17 May 2023   Prob (F-statistic):              0.843
Time:                        23:26:27   Log-Likelihood:                -485.98
No. Observations:                 159   AIC:                             976.0
Df Residuals:                     157   BIC:                             982.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          10.1214      2.826     

In [11]:
stargazer = Stargazer([reg2, reg3, reg4, reg5, reg6, reg7])

In [12]:
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Intercept & 7.078$^{***}$ & 0.862$^{}$ & 22.706$^{***}$ & 45.383$^{***}$ & 21.088$^{***}$ & 20.987$^{***}$ \\
  & (2.708) & (13.633) & (2.967) & (8.271) & (3.092) & (2.931) \\
 animals_val & & & & & & 0.032$^{***}$ \\
  & & & & & & (0.012) \\
 cash_val & & & & & -0.001$^{***}$ & -0.001$^{***}$ \\
  & & & & & (0.000) & (0.000) \\
 diversity_score & 0.412$^{**}$ & 1.201$^{}$ & & & & \\
  & (0.177) & (1.706) & & & & \\
 diversity_squared & & -0.025$^{}$ & & & & \\
  & & (0.053) & & & & \\
 entropy_score & & & -5.587$^{***}$ & -37.489$^{***}$ & -5.766$^{***}$ & -6.239$^{***}$ \\
  & & & (1.694) & (11.022) & (1.770) & (1.696) \\
 entropy_squared & & & & 10.713$^{***}$ & & \\
  & & & & (3.659) & & \\
 improved & -0.031$^{***}$ & -0.031$^{***}$ & 